In [1]:
import numpy as np
import gym

In [21]:
from gym import wrappers
def running_average(x, window_size, mode='valid'):
    return np.convolve(x, np.ones(window_size) / window_size, mode=mode).max()

def check_solution(env, policy, n_episodes = 100, max_steps = 100, to_wrap = False, to_send = False, name2save = ''):
    ns = env.observation_space.n
    na = env.action_space.n
    count_dones = np.zeros(n_episodes)
    count_steps = np.zeros(n_episodes)
    if to_wrap:
        env = wrappers.Monitor(env, name2save)
        
    for i in range(n_episodes):
        observation = env.reset() 
        for step in range(max_steps): 
            action = policy[observation].argmax()
            observation, reward, done, info = env.step(action)
            count_dones[i] += reward
            count_steps[i] += 1
            if done:
                break
    

    env.close()
    if to_wrap and to_send:
        gym.upload(name2save, api_key='sk_bExD4VfCSQukGlQkYKBhdQ')
        
    return running_average(count_dones, 100).max()

In [4]:
def select_a_with_epsilon_greedy(curr_s, q_value, epsilon=0.1):
    a = np.argmax(q_value[curr_s, :])
    if np.random.rand() < epsilon:
        a = np.random.randint(q_value.shape[1])
    return a

In [20]:
def get_greedy_policy(Q):
    ns, na = Q.shape
    policy = np.zeros((ns, na))
    best_actions = Q.argmax(axis = 1)
    policy[np.arange(ns), best_actions] = 1 
    return policy

In [393]:
def dyna_q(env, n_episodes = 1000, n_repeats = 10, gamma = 0.999, alpha = 0.1, eps = 0.9, eps_decay = 0.995, kappa = 0.01):
    max_steps = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    
    ns = env.observation_space.n
    na = env.action_space.n
    
    Q = np.zeros((ns, na))
    R = np.zeros((ns, na))
    S = np.zeros((ns, na))
    
    
    history_length = np.zeros(n_episodes)
    history_reward = np.zeros(n_episodes)
    avg_reward = None
    visited = {}
    for i_episode in range(n_episodes):
        s = env.reset()
        for t in range(max_steps):
            a = select_a_with_epsilon_greedy(s, Q, eps)
            
            if s not in visited.keys():
                visited[s] = set()
            visited[s].add(a)

            
            new_s, reward, done, info = env.step(a)
            new_a = Q[new_s].argmax()
            Q[s][a] += alpha * (reward + gamma * Q[new_s, new_a] - Q[s][a])
            
            history_length[i_episode] = t
            history_reward[i_episode] += reward * gamma ** t
            
            for repeat in range(n_repeats):
                s = np.random.choice(list(visited.keys()))
                a = np.random.choice(tuple(visited[s]))
                reward = R[s, a]
                s1  = S[s, a]
                new_a = Q[s1].argmax()
                Q[s][a] += alpha * (reward + gamma * Q[s1, new_a] - Q[s][a])
                
            s = new_s
            
            #eps *= eps_decay
            if done:
                # Running average of the terminal reward, which is used for controlling an exploration rate
                # (This idea of controlling exploration rate by the terminal reward is suggested by JKCooper2)
                # See https://gym.openai.com/evaluations/eval_xSOlwrBsQDqUW7y6lJOevQ
                if avg_reward == None:
                    avg_reward = reward
                else:
                    avg_reward = kappa * reward + (1 - kappa) * avg_reward
                if reward > avg_reward:
                    # Bias the current policy toward exploitation
                    eps *= eps_decay
            if done:
                break
    return history_length, get_greedy_policy(Q)

In [402]:
def q_learning(env, n_episodes = 1000, gamma = 0.999, alpha = 0.1, eps = 0.9, eps_decay = 0.995, kappa = 0.01):
    max_steps = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    
    ns = env.observation_space.n
    na = env.action_space.n
    
    Q = np.zeros((ns, na))
    
    history_length = np.zeros(n_episodes)
    history_reward = np.zeros(n_episodes)
    avg_reward = None
    for i_episode in range(n_episodes):
        s = env.reset()
        for t in range(max_steps):
            a = select_a_with_epsilon_greedy(s, Q, eps)
            new_s, reward, done, info = env.step(a)
            new_a = Q[new_s].argmax()
            Q[s][a] += alpha * (reward + gamma * Q[new_s, new_a] - Q[s][a])
            
            history_length[i_episode] = t
            history_reward[i_episode] += reward * gamma ** t
            
            s = new_s
            
            #eps *= eps_decay
            if done:
                # Running average of the terminal reward, which is used for controlling an exploration rate
                # (This idea of controlling exploration rate by the terminal reward is suggested by JKCooper2)
                # See https://gym.openai.com/evaluations/eval_xSOlwrBsQDqUW7y6lJOevQ
                if avg_reward == None:
                    avg_reward = reward
                else:
                    avg_reward = kappa * reward + (1 - kappa) * avg_reward
                if reward > avg_reward:
                    # Bias the current policy toward exploitation
                    eps *= eps_decay

            if done:
                break
    return get_greedy_policy(Q)

In [404]:
lake_env = gym.make('FrozenLake8x8-v0')

[2017-07-23 10:07:24,152] Making new env: FrozenLake8x8-v0


In [430]:
%%time
np.random.seed(42)
lens, policy = dyna_q(lake_env, n_episodes = 1000, n_repeats=100,
                    eps = 0.7, eps_decay = 0.995,
                    alpha = 0.1)

/home/liza/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/liza/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


CPU times: user 1min 21s, sys: 16 ms, total: 1min 21s
Wall time: 1min 21s


In [431]:
check_solution(lake_env, policy, n_episodes=1000, max_steps=250)

0.0

In [415]:
%%time
np.random.seed(42)
policy = q_learning(lake_env, n_episodes = 20000, 
                    eps = 0.7, eps_decay = 0.995,
                    alpha = 0.8)

CPU times: user 14.3 s, sys: 0 ns, total: 14.3 s
Wall time: 14.3 s


In [419]:
check_solution(lake_env, policy, n_episodes=1000, max_steps=250)

0.95999999999999996